Errors and how to deal with it

500 internal error - most likely incompatible package, pip uninstall recent package. eg: jupyter-contrib-nbextensions-0.5.1 - is giving 500

*Load libraries

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import s3fs
import os
import numpy as np
%matplotlib inline

In [ ]:
!pip install pyarrow

Connect to EMR and download LABELED data

In [ ]:
#setup imports and paths
import os
import sys
from os.path import expanduser


HOME_DIR = expanduser("~")
sys.path.append(HOME_DIR+'/packages')

In [ ]:
#load custom Midas tools
from Midas import Midas_helper
helper = Midas_helper()

In [ ]:
s3 = s3fs.S3FileSystem()

pq_dataset = pq.ParquetDataset('s3://algorithms-special-access/midas/midas_labeled_data_Q12018', filesystem=s3)
df = pq_dataset.read_pandas().to_pandas()

In [ ]:
df.head()

Convert to csv and save for future analysis

In [ ]:
#save df
helper.cd_main_data()
csv_name = 'midas_labeled_data_Q12018.csv'

In [ ]:
df.to_csv(csv_name, index=None)

Reload csv

In [ ]:

df = pd.read_csv(csv_name)

In [ ]:
df.head()

Column Names

In [ ]:
df.columns

What are the unique vales in 'midas_final_unstructured'

In [ ]:
df['midas_final_unstructured'].unique()

Separate out just the scores and note - other info are irrelevent 
**note the edited COLUMN NAMES

In [ ]:
df_midas_note = df[['midas_final_unstructured', 'cleaned_note_unstructured']]
df_midas_note.columns = ['midas_final', 'cleaned_note']
df_midas_note.head()

In [ ]:
df_midas_note['midas_final'].fillna('Invalid', inplace=True)
df_midas_note.head()

In [ ]:
def plot_value_counts(df_ori, col_to_count, new_col, kind_of_chart):
    df = df_ori[col_to_count].value_counts()
    df = df.to_frame(new_col)
    df[col_to_count] = df.index.values
    df.set_index(col_to_count).T.plot(kind=kind_of_chart, stacked=True)
    return df

In [ ]:
midas_count = plot_value_counts(df_midas_note, 'midas_final', 'Count', 'bar')

In [ ]:
midas_count

Make the classification binary 0 = no midas score 1 = midas score found

In [ ]:
def map_binary_values(row):
    return 0 if row == 'Invalid' else 1

In [ ]:
midas_note_binary = df_midas_note.copy(deep=True)

In [ ]:
midas_note_binary['midas_final'] = midas_note_binary['midas_final'].apply(map_binary_values)
midas_note_binary.head()

In [ ]:
midas_note_binary.to_csv('midas_note_binary.csv')

In [ ]:
midas__binary_count = plot_value_counts(midas_note_binary, 'midas_final', 'Count', 'bar')

In [ ]:
def map_word_num(row):
    return len(row.split())

In [ ]:
midas_note_binary_word_ct = midas_note_binary.copy(deep=True)
midas_note_binary_word_ct['word_count'] = midas_note_binary_word_ct['cleaned_note'].apply(map_word_num)
midas_note_binary_word_ct.head()

In [ ]:
midas_score_word_count = midas_note_binary_word_ct[['midas_final', 'word_count']]
midas_score_word_count.head()

In [ ]:
midas_score_word_count.plot(x='midas_final', y='word_count', kind='scatter')

In [ ]:
midas_score_word_count['word_count'].max()

In [ ]:
print('I am legend'.split())

In [ ]:
ls

In [ ]:
from IPython.display import FileLinks

In [ ]:
ls